In [1]:
import math
import numpy as np
import pandas as pd
import random
import re
import torch
import urllib.request
from torch.utils.data import DataLoader, Dataset
from transformers import PreTrainedTokenizerFast
import urllib.request
import zipfile
import os


In [ ]:
import zipfile

In [ ]:
folder ='/content/drive/MyDrive/01.데이터/2.Validation/원천데이터'

In [ ]:
def unzip_all_files_in_folder(folder_path, extract_to_base):
    # 폴더 내의 모든 파일들을 순회
    for file_name in os.listdir(folder_path):
        # 파일의 전체 경로를 생성
        file_path = os.path.join(folder_path, file_name)

        # ZIP 파일인지 확인
        if zipfile.is_zipfile(file_path):
            # ZIP 파일의 이름을 따서 추출할 디렉토리를 생성
            extract_to = os.path.join(extract_to_base, os.path.splitext(file_name)[0])

            # 디렉토리가 존재하지 않으면 생성
            if not os.path.exists(extract_to):
                os.makedirs(extract_to)

            # ZIP 파일을 읽고 압축 해제
            with zipfile.ZipFile(file_path, 'r') as zip_ref:
                zip_ref.extractall(extract_to)
                print(f'{file_name} 파일이 {extract_to} 경로로 성공적으로 추출되었습니다.')

# 사용 예제
folder_path = '/content/drive/MyDrive/01.데이터/2.Validation/원천데이터'  # ZIP 파일들이 있는 폴더 경로
extract_to_base = '/content/drive/MyDrive/01.데이터/2.Validation/원천데이터'  # 파일을 추출할 기본 디렉토리 경로

unzip_all_files_in_folder(folder_path, extract_to_base)

VS_02. FACEBOOK.zip 파일이 /content/drive/MyDrive/01.데이터/2.Validation/원천데이터/VS_02. FACEBOOK 경로로 성공적으로 추출되었습니다.
VS_04. BAND.zip 파일이 /content/drive/MyDrive/01.데이터/2.Validation/원천데이터/VS_04. BAND 경로로 성공적으로 추출되었습니다.
VS_03. INSTAGRAM.zip 파일이 /content/drive/MyDrive/01.데이터/2.Validation/원천데이터/VS_03. INSTAGRAM 경로로 성공적으로 추출되었습니다.
VS_05. NATEON.zip 파일이 /content/drive/MyDrive/01.데이터/2.Validation/원천데이터/VS_05. NATEON 경로로 성공적으로 추출되었습니다.
VS_01. KAKAO.zip 파일이 /content/drive/MyDrive/01.데이터/2.Validation/원천데이터/VS_01. KAKAO 경로로 성공적으로 추출되었습니다.


In [3]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 

In [1]:
import os
from datasets import Dataset
from tqdm import tqdm
import time
from datasets import load_dataset
import pandas as pd

In [ ]:
folder=['/content/drive/MyDrive/01.데이터/2.Validation/원천데이터/VS_01. KAKAO',
        '/content/drive/MyDrive/01.데이터/2.Validation/원천데이터/VS_02. FACEBOOK',
        '/content/drive/MyDrive/01.데이터/2.Validation/원천데이터/VS_03. INSTAGRAM',
        '/content/drive/MyDrive/01.데이터/2.Validation/원천데이터/VS_04. BAND',
        '/content/drive/MyDrive/01.데이터/2.Validation/원천데이터/VS_05. NATEON']

In [ ]:
data = {"Q": [], "A": [], "id": []}

In [ ]:
id_counter = 1
for folder_path in folder:
    for file in tqdm(os.listdir(folder_path)):
        if file.endswith(".txt"):
            file_path = os.path.join(folder_path, file)
            with open(file_path, "r", encoding="utf-8") as f:
                lines = f.readlines()
                prev_line = ""
                prev_label = None
                for line in lines:
                    line = line.strip()
                    if line:
                        try:
                            label = int(line.split(":")[0])
                            text = line[len(str(label)) + 3 :].strip()
                            if prev_line == "":
                                prev_line = text
                            else:
                                data["id"].append(id_counter)
                                data["Q"].append(prev_line)
                                data["A"].append(text)
                                id_counter += 1
                                prev_line = ""
                        except ValueError:
                            continue


# 데이터셋 생성
dataset = Dataset.from_dict(data)

# 데이터셋을 CSV 파일로 저장
df = pd.DataFrame(data)
csv_filename = "makedata.csv"
df.to_csv(csv_filename, index=False, encoding="utf-8")

print(dataset[0])
len(dataset)

100%|██████████| 201/201 [00:02<00:00, 90.32it/s] 


{'Q': '오늘 저녁에 오랜만에 영화 고고?', 'A': '오우 너무 좋지? ㅎ', 'id': 1}


In [4]:
!pip install pytorch_lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.3/812.3 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 15.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-many

In [2]:
from pytorch_lightning import LightningModule

In [21]:
import numpy as np
import pandas as pd
import torch
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint

from torch.utils.data import DataLoader, Dataset
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel
import re, os
from tqdm import tqdm


Q_TKN = "<usr>"
A_TKN = "<sys>"
BOS = "</s>"
EOS = "</s>"
MASK = "<unused0>"
SENT = "<unused1>"
PAD = "<pad>"

save_dir = "saved_models"
os.makedirs(save_dir, exist_ok=True)


print("start1")


class ChatbotDataset(Dataset):
    def __init__(self, chats, max_len=40):  # 데이터셋의 전처리를 해주는 부분
        self._data = chats
        self.max_len = max_len
        self.q_token = Q_TKN
        self.a_token = A_TKN
        self.sent_token = SENT
        self.eos = EOS
        self.mask = MASK
        self.tokenizer = koGPT2_TOKENIZER

    def __len__(self):  # chatbotdata 의 길이를 리턴한다.
        return len(self._data)

    def __getitem__(self, idx):  # 로드한 챗봇 데이터를 차례차례 DataLoader로 넘겨주는 메서드
        turn = self._data.iloc[idx]
        q = turn["Q"]  # 질문을 가져온다.
        q = re.sub(r"([?.!,])", r" ", q)  # 구둣점들을 제거한다.

        a = turn["A"]  # 답변을 가져온다.
        a = re.sub(r"([?.!,])", r" ", a)  # 구둣점들을 제거한다.

        q_toked = self.tokenizer.tokenize(self.q_token + q + self.sent_token)
        q_len = len(q_toked)

        a_toked = self.tokenizer.tokenize(self.a_token + a + self.eos)
        a_len = len(a_toked)

        # 질문의 길이가 최대길이보다 크면
        if q_len > self.max_len:
            a_len = self.max_len - q_len  # 답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:  # 질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]  # 질문길이를 최대길이의 반으로
                q_len = len(q_toked)
                a_len = self.max_len - q_len  # 답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        # 질문의 길이 + 답변의 길이가 최대길이보다 크면
        if q_len + a_len > self.max_len:
            a_len = self.max_len - q_len  # 답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:  # 질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]  # 질문길이를 최대길이의 반으로
                q_len = len(q_toked)
                a_len = self.max_len - q_len  # 답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        # 답변 labels = [mask, mask, ...., mask, ..., <bos>,..답변.. <eos>, <pad>....]
        labels = [
            self.mask,
        ] * q_len + a_toked[1:]

        # mask = 질문길이 0 + 답변길이 1 + 나머지 0
        mask = [0] * q_len + [1] * a_len + [0] * (self.max_len - q_len - a_len)
        # 답변 labels을 index 로 만든다.
        labels_ids = self.tokenizer.convert_tokens_to_ids(labels)
        # 최대길이만큼 PADDING
        while len(labels_ids) < self.max_len:
            labels_ids += [self.tokenizer.pad_token_id]

        # 질문 + 답변을 index 로 만든다.
        token_ids = self.tokenizer.convert_tokens_to_ids(q_toked + a_toked)
        # 최대길이만큼 PADDING
        while len(token_ids) < self.max_len:
            token_ids += [self.tokenizer.pad_token_id]

        # 질문+답변, 마스크, 답변
        return (token_ids, np.array(mask), labels_ids)


def collate_batch(batch):
    data = [item[0] for item in batch]
    mask = [item[1] for item in batch]
    label = [item[2] for item in batch]
    return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)


koGPT2_TOKENIZER = PreTrainedTokenizerFast.from_pretrained(
    "skt/kogpt2-base-v2",
    bos_token=BOS,
    eos_token=EOS,
    unk_token="<unk>",
    pad_token=PAD,
    mask_token=MASK,
)
model = GPT2LMHeadModel.from_pretrained("skt/kogpt2-base-v2")

dataname = "/content/makedata.csv"  # Adjusted file path
Chatbot_Data = pd.read_csv(dataname)
Chatbot_Data = Chatbot_Data[:100]
Chatbot_Data.dropna(subset=["A"], inplace=True)
Chatbot_Data.dropna(subset=["Q"], inplace=True)
Chatbot_Data.dropna(subset=["id"], inplace=True)


print("start3")


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_set = ChatbotDataset(Chatbot_Data, max_len=40)
train_dataloader = DataLoader(
    train_set,
    batch_size=32,
    num_workers=0,
    shuffle=True,
    collate_fn=collate_batch,
)

model.to(device)

learning_rate = 3e-5
criterion = torch.nn.CrossEntropyLoss(reduction="none")
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

num_epochs = 3
Sneg = -1e18

for epoch in range(num_epochs):
    dataloader = tqdm(train_dataloader, desc=f"Epoch {epoch}")
    for batch_idx, samples in enumerate(dataloader):
        optimizer.zero_grad()
        token_ids, mask, label = samples
        token_ids, mask, label = token_ids.to(device), mask.to(device), label.to(device)
        out = model(token_ids)
        out = out.logits
        mask_3d = mask.unsqueeze(dim=2).repeat_interleave(repeats=out.shape[2], dim=2)
        mask_out = torch.where(mask_3d == 1, out, Sneg * torch.ones_like(out))
        loss = criterion(mask_out.transpose(2, 1), label)
        print(out.shape)
        print(label.shape)
        print(mask.shape)
        print(mask_3d.shape)
        print(mask_out.shape)
        print(mask_out.transpose(2, 1).shape)
        print(label.shape)
        avg_loss = loss.sum() / mask.sum()
        print('avg_loss', avg_loss)
        avg_loss.backward()
        optimizer.step()

model_save_path = os.path.join(save_dir, "chatbot_model.pth")
torch.save(
    {
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "epoch": epoch,
    },
    model_save_path,
)

print("Model saved at:", model_save_path)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


start1
start3


Epoch 0:  25%|██▌       | 1/4 [00:00<00:00,  4.20it/s]

torch.Size([32, 40, 51200])
torch.Size([32, 40])
torch.Size([32, 40])
torch.Size([32, 40, 51200])
torch.Size([32, 40, 51200])
torch.Size([32, 51200, 40])
torch.Size([32, 40])
avg_loss tensor(27.6917, device='cuda:0', grad_fn=<DivBackward0>)


Epoch 0:  50%|█████     | 2/4 [00:00<00:00,  3.32it/s]

...

Epoch 2:  75%|███████▌  | 3/4 [00:00<00:00,  3.35it/s]

torch.Size([32, 40, 51200])
torch.Size([32, 40])
torch.Size([32, 40])
torch.Size([32, 40, 51200])
torch.Size([32, 40, 51200])
torch.Size([32, 51200, 40])
torch.Size([32, 40])
avg_loss tensor(24.3373, device='cuda:0', grad_fn=<DivBackward0>)


Epoch 2: 100%|██████████| 4/4 [00:01<00:00,  3.53it/s]

torch.Size([4, 40, 51200])
torch.Size([4, 40])
torch.Size([4, 40])
torch.Size([4, 40, 51200])
torch.Size([4, 40, 51200])
torch.Size([4, 51200, 40])
torch.Size([4, 40])
avg_loss tensor(29.6866, device='cuda:0', grad_fn=<DivBackward0>)


Model saved at: saved_models/chatbot_model.pth


In [ ]:
# print('out',out.shape)
#         print(label.shape)
#         print(mask.shape)
#         print(mask_3d.shape)
#         print(mask_out.shape)
#         print(mask_out.transpose(2, 1).shape)
#         print(label.shape)

In [9]:
token_ids
token_ids.shape
out.shape

torch.Size([4, 40, 51200])

In [10]:
label.shape

torch.Size([4, 40])

In [13]:
mask_3d.shape

torch.Size([4, 40, 51200])

In [15]:
mask_out.shape

torch.Size([4, 40, 51200])

In [17]:
mask_out.transpose(2, 1).shape

torch.Size([4, 51200, 40])

In [16]:
mask.shape

torch.Size([4, 40])

In [12]:
train_set[0]


([2,
  10070,
  44596,
  12474,
  20486,
  10584,
  17577,
  739,
  10,
  4,
  9114,
  8092,
  12371,
  18364,
  739,
  739,
  608,
  1,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3],
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 [9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9114,
  8092,
  12371,
  18364,
  739,
  739,
  608,
  1,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3])

In [ ]:
import torch
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel

# Define your special tokens and tokenizer
Q_TKN = "<usr>"
A_TKN = "<sys>"
SENT = "<unused1>"
EOS = "</s>"
BOS = "</s>"

# Initialize your tokenizer
koGPT2_TOKENIZER = PreTrainedTokenizerFast.from_pretrained(
    "skt/kogpt2-base-v2",
    bos_token=BOS,
    eos_token=EOS,
    unk_token="<unk>",
    pad_token="<pad>",
    mask_token="<unused0>",
)

# Load your trained model
model_path = "saved_models/chatbot_model.pth"  # Adjust the path to your saved model
model = GPT2LMHeadModel.from_pretrained("skt/kogpt2-base-v2")
checkpoint = torch.load(model_path)
model.load_state_dict(checkpoint["model_state_dict"])
model.eval()

# Interaction loop with the chatbot
with torch.no_grad():
    while 1:
        q = input("user > ").strip()
        if q == "quit":
            break
        a = ""
        while 1:
            input_ids = torch.LongTensor(
                koGPT2_TOKENIZER.encode(Q_TKN + q + SENT + A_TKN + a)
            ).unsqueeze(dim=0)
            pred = model(input_ids)
            pred = pred.logits
            gen = koGPT2_TOKENIZER.convert_ids_to_tokens(
                torch.argmax(pred, dim=-1).squeeze().numpy().tolist()
            )[-1]
            if gen == EOS:
                break
            a += gen.replace("▁", " ")
        print("Chatbot > {}".format(a.strip()))

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


user > 범죄도시 재밌냐?
Chatbot > 나 그런 거 안 봤어
user > 범죄 도시 같은 영화 추천 해줘
Chatbot > 나 범죄 도시  제일 최근에 본 영화야
user > 다른 영화 없어?
Chatbot > 나 외국 영화 좋아하잖아~
user > 한국영화는 안 좋아하냐?
Chatbot > 응응 영화가 제일 재밌어
user > 어제는 모 했어?
Chatbot > 오늘부터인가
user > 아침 밥은 먹었어?
Chatbot > 응 근데 나 오메가쓰리 많이 먹어서 요즘 오메가쓰리 많이 먹는 중
user > 개킹받네
Chatbot > 그럼 그 강아지 쌉가능
user > 내놔 임마
Chatbot > 그럼 너가 대신 들어가 줄게
user > ㅋㅋㅋ
Chatbot > 아유 키키 그래도 이번 추석은 잘 버틸 수 있겠어


In [ ]:
# 1 데이터 더 찾기
# 2 데이터 증식
# 3 레그?? 어떻게 쓰는지 찾아보기
# 4 파이토치 공부하기
# 4 프론트 엔지니어링으로 대화 톤 바꾸기 ex)고분고분, 겸손

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.layer1=nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kerner_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

In [ ]:
import torchmetrics
preds=torch.rand(10,5).softmax(dim=1)
labels=torch.randint(5,(10,))

acc=torchmetrics.functional.accuracy(preds, target)